# Reconocimiento personalizado de celebridades usando Amazon Rekognition

In [ ]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os
from io import BytesIO

In [ ]:
rekognition = boto3.client('rekognition')
dynamodb = boto3.client('dynamodb')
s3 = boto3.client('s3')

## DynamoDB table to store custom celebrity metadata

In [ ]:
ddbTableName = "my-celebrities"
collectionId = "my-celebrities"

In [ ]:
listTablesResponse = dynamodb.list_tables()
display(listTablesResponse["TableNames"])

In [ ]:
createTableResponse = dynamodb.create_table(
    TableName=ddbTableName,
    KeySchema=[
        {
            'AttributeName': 'id',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'id',
            'AttributeType': 'S'
        },

    ],
    BillingMode='PAY_PER_REQUEST'
)

display(createTableResponse)

In [ ]:
listTablesResponse = dynamodb.list_tables()
display(listTablesResponse["TableNames"])

## Rekogniton Collection para almacenar rostros

In [ ]:
listCollectionsResponse = rekognition.list_collections()
display(listCollectionsResponse["CollectionIds"])
display(listCollectionsResponse["FaceModelVersions"])

In [ ]:
createCollectionResponse = rekognition.create_collection(
    CollectionId=collectionId
)
display(createCollectionResponse)

In [ ]:
listCollectionsResponse = rekognition.list_collections()

display(listCollectionsResponse["CollectionIds"])
display(listCollectionsResponse["FaceModelVersions"])

In [ ]:
describeCollectionResponse = rekognition.describe_collection(
    CollectionId=collectionId
)
display(describeCollectionResponse)

# Indexacion de rostros de celebridades personalizado

In [ ]:
bucketName = "{REPLACE_YOUR_BUCKET}"
s3.upload_file(
    Filename="resources/kashif01.jpg",
    Bucket=bucketName,
    Key="media/kashif01.jpg",
)
s3.upload_file(
    Filename="resources/kashif02.jpg",
    Bucket=bucketName,
    Key="media/kashif02.jpg",
)
s3.upload_file(
    Filename="resources/kashif03.jpg",
    Bucket=bucketName,
    Key="media/kashif03.jpg",
)
s3.upload_file(
    Filename="resources/chris01.png",
    Bucket=bucketName,
    Key="media/chris01.png",
)
s3.upload_file(
    Filename="resources/chris02.png",
    Bucket=bucketName,
    Key="media/chris02.png",
)
s3.upload_file(
    Filename="resources/chris03.png",
    Bucket=bucketName,
    Key="media/chris03.png",
)


In [ ]:
def indexFace (bucketName, imageName, celebrityId):

    indexFaceResponse = rekognition.index_faces(
        CollectionId=collectionId,
        Image={
            'S3Object': {
                'Bucket': bucketName,
                'Name': imageName,
            }
        },
        ExternalImageId=celebrityId,
        DetectionAttributes=[
            'DEFAULT' #'DEFAULT'|'ALL',
        ],
        MaxFaces=1,
        QualityFilter='AUTO' #NONE | AUTO | LOW | MEDIUM | HIGH
    )
    
    display(indexFaceResponse)

def addCelebrityToDynamoDB(celebrityId, celebrityName, celebrityUrl):
    ddbPutItemResponse = dynamodb.put_item(
        Item={
            'id': {'S': celebrityId},
            'name': {'S': celebrityName},
            'url': { 'S': celebrityUrl},
        },
        TableName=ddbTableName,
    )

In [ ]:
# Indexacion de la celebridad 1
celebrityId = "1"
celebrityName = "Chris Munns"
celebrityUrl = "http://www.amazon.com"

In [ ]:
addCelebrityToDynamoDB(celebrityId, celebrityName, celebrityUrl)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': "media/chris01.png"})))

In [ ]:
indexFace(bucketName, "media/chris01.png", celebrityId)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': "media/chris02.png"})))

In [ ]:
indexFace(bucketName, "media/chris02.png", celebrityId)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': "media/chris03.png"})))

In [ ]:
indexFace(bucketName, "media/chris03.png", celebrityId)

In [ ]:
describeCollectionResponse = rekognition.describe_collection(
    CollectionId=collectionId
)
display("FaceCount: {0}".format(describeCollectionResponse["FaceCount"]))

In [ ]:
# Indexacion de la celebridad 2
celebrityId = "2"
celebrityName = "Kashif Imran"
celebrityUrl = "http://aws.amazon.com"

In [ ]:
addCelebrityToDynamoDB(celebrityId, celebrityName, celebrityUrl)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': "media/kashif01.jpg"})))

In [ ]:
indexFace(bucketName, "media/kashif01.jpg", celebrityId)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': "media/kashif02.jpg"})))

In [ ]:
indexFace(bucketName, "media/kashif02.jpg", celebrityId)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': "media/kashif03.jpg"})))

In [ ]:
indexFace(bucketName, "media/kashif03.jpg", celebrityId)

In [ ]:
describeCollectionResponse = rekognition.describe_collection(
    CollectionId=collectionId
)
display("FaceCount: {0}".format(describeCollectionResponse["FaceCount"]))

In [ ]:
# Subimos la imagen a reconocer
imageName='media/serverless-bytes.png'
s3.upload_file(
    Filename="resources/serverless-bytes.png",
    Bucket=bucketName,
    Key=imageName,
)

In [ ]:
searchFacesResponse = rekognition.search_faces_by_image(
    CollectionId=collectionId,
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    },
    MaxFaces=2,
    FaceMatchThreshold=95
)

In [ ]:
display(searchFacesResponse)

In [ ]:
def displayWithBoundingBoxes (sourceImage, boxes):
    colors = ((220,220,220),(230,230,230),(76,182,252),(52,194,123))
    
    imageLocation = 'resources/'+os.path.basename(sourceImage)

    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
       imageFormat = 'JPEG'

    bbImage.save(imageLocation,format=imageFormat)

    display(bbImage)
    
def getDynamoDBItem(itemId):
    ddbGetItemResponse = dynamodb.get_item(
        Key={'id': {'S': itemId} },
        TableName=ddbTableName
    )
    
    itemToReturn = ('', '', '')
    
    if('Item' in ddbGetItemResponse):
        itemToReturn = (ddbGetItemResponse['Item']['id']['S'], 
                ddbGetItemResponse['Item']['name']['S'],
                ddbGetItemResponse['Item']['url']['S'])
    
    return itemToReturn

In [ ]:
def displaySearchedFace(sfr):  

    boxes = []
    
    if(len(sfr['FaceMatches']) > 0):
        bb = sfbb = sfr['SearchedFaceBoundingBox']
        eid = sfr['FaceMatches'][0]['Face']['ExternalImageId']
        conf = sfr['FaceMatches'][0]['Similarity']

        celeb = getDynamoDBItem(eid)

        boxes.append(("{0}-{1}-{2}%".format(celeb[0], celeb[1], round(conf,2)), bb))

        displayWithBoundingBoxes(imageName, boxes)

displaySearchedFace(searchFacesResponse)

## Reconocer todas las celebridades personalizadas en la imagen

In [ ]:
def detectFaces():
    detectFacesResponse = rekognition.detect_faces(
        Image={
            'S3Object': {
                'Bucket': bucketName,
                'Name': imageName
                }
            },
        Attributes=['DEFAULT'])
    return detectFacesResponse

In [ ]:
def getFaceCrop(imageBinary, box, image_width, image_height):
    
    x1 = int(box['Left'] * image_width)-25
    y1 = int(box['Top'] * image_height)-25
    x2 = int(box['Left'] * image_width + box['Width'] * image_width)+25
    y2 = int(box['Top'] * image_height + box['Height']  * image_height)+25
    if x1 < 0 : x1=0
    if y1 < 0 : y1=0
    if x2 < 0 : x2=image_width
    if y2 < 0 : y2=image_height 
           
    coordinates = (x1,y1,x2,y2)

    image_crop = imageBinary.crop(coordinates)
    stream2 = BytesIO() 
    
    iformat = "JPEG"
    if(imageName.lower().endswith("png")):
        iformat = "PNG"
    
    image_crop.save(stream2,format=iformat)
    image_region_binary = stream2.getvalue()    
    stream2.close()
    
    return image_region_binary

In [ ]:
def recognizeFace(faceCrop):
    searchFacesResponse = rekognition.search_faces_by_image(
        CollectionId=collectionId,
        Image={
            'Bytes': faceCrop
            },
        MaxFaces=2,
        FaceMatchThreshold=95
    )

    if(len(searchFacesResponse['FaceMatches']) > 0):
        eid = searchFacesResponse['FaceMatches'][0]['Face']['ExternalImageId']
        conf = searchFacesResponse['FaceMatches'][0]['Similarity']
        celeb = getDynamoDBItem(eid)

        return "{0}-{1}-{2}%".format(celeb[0], celeb[1], round(conf,2))
    else:
        return ""


In [ ]:
def recognizeAllCustomCelebrities():
    detectedFaces = detectFaces()
    
    imageLocation = 'resources/'+os.path.basename(imageName)
    
    imageBinary = Image.open(imageLocation)
    width, height = imageBinary.size 
    
    boxes = []
    for detectedFace in detectedFaces['FaceDetails']:
        faceCrop = getFaceCrop(imageBinary, detectedFace['BoundingBox'], width, height)
        recognizedFace = recognizeFace(faceCrop)
        if(recognizedFace):
            boxes.append((recognizedFace, detectedFace['BoundingBox']))
        else:
            boxes.append(("Unrecognized Face", detectedFace['BoundingBox']))
    displayWithBoundingBoxes(imageName, boxes)

In [ ]:
recognizeAllCustomCelebrities()